## Analyze whether SNWD varies more from year to year or from place to place.

In [22]:
import pandas as pd
import numpy as np
import urllib
import math

In [23]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [24]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [25]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SSSBSSBB'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
420
+-------------------+-----------------+-------------------+-----------------+---------+--------+--------+---------+-----------+-------------------+-------------------+-------------------+--------------------+------------------+-----------+------------+------+--------------------+------+
|            coeff_1|          coeff_2|            coeff_3|          coeff_4|elevation|   label|latitude|longitude|measurement|              res_1|              res_2|              res_3|               res_4|          res_mean|    station|   total_var|undefs|              vector|  year|
+-------------------+-----------------+-------------------+-----------------+---------+--------+--------+---------+-----------+-------------------+-------------------+-------------------+--------------------+------------------+-----------+------------+------+--------------------+------+
|-12268.765203613644|825.7982574517614| 2286.6759004873757|236

In [26]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'coeff_4', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_4', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [34]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_3'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_3 FROM weather


station    year      coeff_3
0  USC00044176  1965.0  2286.675900
1  USC00049633  1973.0 -1863.413671
2  USC00044176  1953.0   901.552775
3  USC00049632  1967.0  1179.851233
4  USC00049632  1958.0  -207.004981

In [35]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_3')
year_station_table.tail(10)

station  USC00040684  USC00040755  USC00040820  USC00040824  USC00041697  \
year                                                                       
2003.0           NaN          NaN          NaN          NaN  1181.885739   
2004.0           NaN          NaN          NaN          NaN   937.011720   
2005.0           NaN          NaN          NaN          NaN  2279.891689   
2006.0           NaN          NaN          NaN          NaN  -259.349650   
2007.0           NaN          NaN          NaN          NaN  -803.684651   
2008.0           NaN          NaN -1841.660972          NaN  -697.037330   
2009.0           NaN          NaN          NaN          NaN  -671.598437   
2010.0           NaN          NaN          NaN          NaN   626.682440   
2011.0           NaN          NaN          NaN          NaN          NaN   
2012.0           NaN          NaN          NaN          NaN          NaN   

station  USC00042331  USC00042539  USC00043939  USC00044170  USC00044176  \
year                                                                       
2003.0           NaN          NaN          NaN          NaN          NaN   
2004.0           NaN          NaN          NaN          NaN          NaN   
2005.0           NaN          NaN          NaN          NaN          NaN   
2006.0           NaN          NaN  -282.922257          NaN          NaN   
2007.0           NaN          NaN          NaN          NaN          NaN   
2008.0           NaN          NaN   346.140974          NaN          NaN   
2009.0           NaN          NaN  -227.516725          NaN          NaN   
2010.0           NaN          NaN   227.972295          NaN          NaN   
2011.0           NaN          NaN          NaN          NaN          NaN   
2012.0           NaN          NaN          NaN          NaN          NaN   

station     ...       USC00049855  USC00260046  USC00260668  USC00261755  \
year        ...                                                            
2003.0      ...        471.763529          NaN          NaN          NaN   
2004.0      ...               NaN          NaN          NaN          NaN   
2005.0      ...               NaN          NaN          NaN          NaN   
2006.0      ...               NaN          NaN          NaN          NaN   
2007.0      ...               NaN          NaN          NaN          NaN   
2008.0      ...               NaN          NaN          NaN          NaN   
2009.0      ...               NaN          NaN          NaN          NaN   
2010.0      ...               NaN          NaN          NaN          NaN   
2011.0      ...       -212.903044          NaN          NaN          NaN   
2012.0      ...               NaN          NaN          NaN          NaN   

station  USC00262431  USC00263285  USC00265362  USC00268160  USW00023153  \
year                                                                       
2003.0           NaN          NaN          NaN          NaN          NaN   
2004.0           NaN          NaN          NaN          NaN          NaN   
2005.0           NaN          NaN          NaN          NaN          NaN   
2006.0           NaN          NaN          NaN          NaN          NaN   
2007.0           NaN          NaN          NaN          NaN          NaN   
2008.0           NaN          NaN          NaN          NaN          NaN   
2009.0           NaN          NaN          NaN          NaN          NaN   
2010.0           NaN          NaN          NaN          NaN          NaN   
2011.0           NaN          NaN          NaN          NaN          NaN   
2012.0           NaN          NaN          NaN          NaN          NaN   

station  USW00023157  
year                  
2003.0           NaN  
2004.0           NaN  
2005.0           NaN  
2006.0           NaN  
2007.0           NaN  
2008.0           NaN  
2009.0           NaN  
2010.0           NaN  
2011.0           NaN  
2012.0           NaN  

[10 rows x 33 columns]

### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [36]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

total RMS                   =  1052.12813966
RMS removing mean-by-station=  1018.94773308
RMS removing mean-by-year   =  694.772844026


In [20]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(10):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 872.516277667
0 after removing mean by year    = 808.870716883
0 after removing mean by stations= 461.858080728
1 after removing mean by year    = 441.695716487
1 after removing mean by stations= 438.450436059
2 after removing mean by year    = 437.51857751
2 after removing mean by stations= 437.154389104
3 after removing mean by year    = 436.982825713
3 after removing mean by stations= 436.889519664
4 after removing mean by year    = 436.832211519
4 after removing mean by stations= 436.793392844
5 after removing mean by year    = 436.765159178
5 after removing mean by stations= 436.743636802
6 after removing mean by year    = 436.726749815
6 after removing mean by stations= 436.713273284
7 after removing mean by year    = 436.702413694
7 after removing mean by stations= 436.693614991
8 after removing mean by year    = 436.686464311
8 after removing mean by stations= 436.680643117
9 after removing mean by year    = 436.675899768
9 after removing mean by stations= 436.6720